In [2]:
import sys; sys.path.append('..')
import torch
from torch.utils.data import random_split
import pandas as pd
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

print("In Colab: {}".format(IN_COLAB))

if IN_COLAB:
  !pip install torchmetrics
  !pip install kornia
  !pip install torchvision
  google.colab.drive.mount('/content/drive')
  %cd /content/drive/My Drive/Go-Viral-Project/notebooks


# Using CNN for feature extraction from audio input

In [5]:
AUDIO_PATH = '../data/audio'

TENSOR_PATH = '../data/specs'

METADATA_PATH = '../data/metadata.csv'

SEED = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(SEED)

In [6]:
# clean up df - remove songs that dont have spectrograms
import os

df = pd.read_csv('../data/metadata.csv')
data_path = '../data/specs/'
files_not_found = 0

for idx, row in df.iterrows():
    song_path = os.path.join(data_path, row['id'] + '.pt')

    if not os.path.exists(song_path):
        # print(f"File not found: {song_path}")
        df = df.drop(idx)
        files_not_found += 1

print(f"Number of files not found: {files_not_found}")

Number of files not found: 0


In [7]:
from torchvision import transforms

def preprocess_mbnet(X):
  prep = transforms.Compose([
      transforms.Resize((224,2206)),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  return prep(torch.cat((X,X,X),dim=0))

In [8]:
from src.RNN_utils.dataset import SoundDS
from torch.utils.data import default_collate

myds = SoundDS(df, '../data/specs/', preprocess_mbnet)

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

to_gpu = lambda x: list(map(lambda t: t.to(device), default_collate(x)))
# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, collate_fn = to_gpu, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, collate_fn = to_gpu, shuffle=False)

In [9]:
b_size, channels, hight, width = next(iter(train_dl))[0].shape
num_batches = len(train_dl)
print(f'num batches: {num_batches}\nbatch size: {b_size}\nchannels: {channels}\nhight: {hight} \nwidth: {width}')

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


num batches: 183
batch size: 16
channels: 3
hight: 224 
width: 2206


## MobileNet V2:

### Overfitting the model:

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.classifier = nn.Sequential(
    nn.Linear(1280,2),
    nn.Softmax(dim=1)
)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

epochs = 20

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
print(model)

In [ ]:
sum(param.numel() for param in model.parameters())

2226434

In [ ]:
(X,y) = next(iter(train_dl))

In [ ]:
for epoch in range(epochs):
    optimizer.zero_grad()
    y_prob = model(X)
    loss = criterion(y_prob,y)
    loss.backward()
    optimizer.step()
    loss = loss.item()
    scheduler.step()
    acc = torch.sum(torch.argmax(y_prob,dim=1)==y).item()/b_size
    print(f'Epoch #{epoch}: Loss - {loss}, Accuracy - {acc}')

### Training the model:

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.classifier = nn.Sequential(
    nn.Linear(1280,2),
    nn.Softmax(dim=1)
)

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.2)

In [ ]:
from src.RNN_utils.trainer import trainer
from src.RNN_utils.cross_val import crossValidate

train_model = trainer(model,criterion,optimizer,scheduler,device)

In [ ]:
results = train_model.train(train_dl,20,True)

### Cross validation:

In [10]:
configs = []
results = []

In [11]:
from src.RNN_utils.trainer import trainer
from src.RNN_utils.cross_val import crossValidate
from kornia.losses.focal import BinaryFocalLossWithLogits

config = {'lr':1e-3, 'weight_decay':1e-4, 'step_size': 10, 'gamma': 0.1}

model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.classifier = nn.Sequential(
    nn.Linear(1280,2),
    nn.Softmax(dim=1)
)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=config['lr'], weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=config['step_size'],gamma=config['gamma'])

train_model = trainer(model,criterion,optimizer,scheduler,device)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 122MB/s]


In [12]:
cv_obj = crossValidate(train_ds=train_ds, device=device, folds=3, batch_size=16)

In [ ]:
results.append(cv_obj.runCV(train_model, epochs=20))
configs.append(config)

Fold #0:


Train Batch:   0%|          | 0/122 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Train Batch: 100%|██████████| 122/122 [32:05<00:00, 15.78s/it]


Epoch #0: Loss - 83.9294042289257, Accuracy - 0.6132852729145211


Test Batch: 100%|██████████| 61/61 [15:26<00:00, 15.19s/it]


Val results: Loss - 42.07582414150238, Accuracy - 0.5534979423868313


...

Train Batch: 100%|██████████| 122/122 [02:00<00:00,  1.01it/s]


Epoch #12: Loss - 79.47807559370995, Accuracy - 0.638188368502316


Test Batch: 100%|██████████| 61/61 [00:36<00:00,  1.67it/s]


Val results: Loss - 40.88321179151535, Accuracy - 0.6127703398558187


Train Batch:  74%|███████▍  | 90/122 [01:29<00:31,  1.02it/s]

In [ ]:
from src.RNN_utils.cross_val import plotCV

plotCV(results, configs,title='Cross Validation for CNN MobileNetV2 with Cross-Entropy Loss')

In [ ]:
print(results)